
# Intelligent Systems

## Assignment 5 - A * Algorithm using Manhattan heuristic - _8 puzzle_

## February 6th 2019, A01227071, Arturo Fornés Arvayo

**Problem Description:** Implement the INFORMED search algorithm A* using Manhattan heuristic by coding. 

_Method:_
Develop a program that runs through a search tree built by the A* algorithm for the 8-puzzle problem.
* The program should read a text file which contains the initial state of the 8-puzzle.
* The target state is always 0 1 2 3 4 5 6 7 8 (i.e., eight digits from 0 to 8 separated by a space, zero represents the space in 8-puzzle).
* Show the content of the HEAP for each level (depth), showing the heuristics in increasing order.


The initial state could be, for example: **7 2 4 5 0 6 8 3 1** and the final state by default is **0 1 2 3 4 5 6 7 8**.

Calculations:

f(n) = h (Manhattan heuristic) + g (depth)

![puzzle](./img/a01.png)

In [1]:
initial_state = "7 2 4 5 0 6 8 3 1"
#initial_state = "1 4 2 3 0 5 6 7 8"
#initial_state = "1 2 5 3 4 0 6 7 8"
side = 3

## Node Class

In [27]:
def to_array(string):
        return list(map(lambda x: int(x), string.split()))
def is_solvable(init):
    inversions = 0

    for i in range(0, len(init)):
        for j in range(i+1, len(init)):
            if init[j] != 0 and init[i] != 0 and init[j] > init[i]:
                inversions += 1

    if inversions % 2 == 1:
        return False
    else:
        return True

class PathNode(object):
    def __init__(self, me, parent, depth, direction):
        self.me = me
        self.parent = parent
        self.depth = depth
        self.direction = direction
        self.priority = depth + self.manhattan_heuristic()

    def __string__(self):
        string = ""
        arr = self.me
        for i in range(len(arr)):
            if i % side == 0:
                string += "\n"
            string += str(arr[i])
        return string
    def __repr__(self):
        string = ""
        arr = self.me
        for i in range(len(arr)):
            if i % side == 0:
                string += "\n"
            string += str(arr[i])
        string += " p(n) = " + str(self.priority)
        return string
    def to_array(self, string):
        return list(map(lambda x: int(x), string.split()))

    def to_string(self):
        return " ".join(map(lambda x: str(x), self.me))
    
    def value(self, row, col):
        return self.me[row*side+col]

    def index(self, row, col):
        return row*side+col
    
    def indexOf(self, value):
        for i in range(len(self.me)):
            if self.me[i] == value:
                return i

    def position(self, index):
        col = index % side
        row = index // side
        return row, col

    def swap(self, x, y, xf, yf):
        copy_arr = list(self.me)
        init_index = self.index(x, y)
        final_index = self.index(xf, yf)
        copy_arr[init_index], copy_arr[final_index] = copy_arr[final_index], copy_arr[init_index]
        return copy_arr
        
    def expand_node(self):
        row, col = self.position(self.indexOf(0))    
        children = []
        if row > 0:
            up = self.swap(row, col, row - 1, col)
            children.append(PathNode(up, self, self.depth + 1, "up"))
        if row < side - 1:
            down = self.swap(row, col, row + 1, col)
            children.append(PathNode(down, self, self.depth + 1, "down"))
        if col > 0:
            left = self.swap(row, col, row, col - 1)
            children.append(PathNode(left, self, self.depth + 1, "left"))
        if col < side - 1:
            right = self.swap(row, col, row, col + 1)
            children.append(PathNode(right, self, self.depth + 1, "right"))
        return children
    
    def manhattan_heuristic(self):
        final_positions = [
            [0, 0],
            [0, 1],
            [0, 2],
            [1, 0],
            [1, 1],
            [1, 2],
            [2, 0],
            [2, 1],
            [2, 2]
        ]
        count = 0
        for i, num in enumerate(self.me):
            if num == 0:
                continue
            row, col = self.position(i)
            count += abs(row - final_positions[num][0])
            count += abs(col - final_positions[num][1])
        return count
    
    def __lt__(self, other):
        return self.priority < other.priority

    def __le__(self, other):
        return self.priority <= other.priority

    def __ge__(self, other):
        return self.priority >= other.priority

    def __gt__(self, other):
        return self.priority > other.priority

## A* Informed Search

In [34]:
def AStar(init, final):
    from queue import PriorityQueue # Priority Queue (Heap) structure
    if not is_solvable(to_array(init)):
        return None, 0

    known = set()
    
    path = {}
    
    init = to_array(init)
    init = PathNode(init, None, 0, "start")
    
    PQ = PriorityQueue()
    PQ.put((init.priority, init))
    
    visited = 0
    
    while len(PQ.queue) > 0:
        print("Heap: ", [entry.me for priority, entry in PQ.queue[:4]])
        _, current_node = PQ.get()
        str_current = current_node.to_string()
        
        visited += 1
        
        if str_current == final:
            return current_node, visited

        known.add(str_current)
        children = current_node.expand_node()
        
        for child in children:
            str_child = child.to_string()
            if str_child not in known:
                known.add(str_child)
                PQ.put((child.priority, child))
    return None, 0

import time
import sys

start = time.time()
node, visited_nodes = AStar(initial_state, "0 1 2 3 4 5 6 7 8")
size = sys.getsizeof(node)
end = time.time()
duration = end - start

if (node is None):
    print("Unsolvable")
else:
    temp_node = node
    print("==============")
    print("Cost to path (depth): ", node.depth)
    print("Visited Nodes: ", visited_nodes)
    print("Used Memory (assuming a node is 72 bytes): ", visited_nodes * 72, "bytes")
    print("Size in bytes of PathNode: ", size)
    print("Actual used memory: ", size * visited_nodes, "bytes")
    
    if duration < 0.010:
        duration *= 1000
        unit = "milliseconds"
    else:
        unit = "seconds"
    print("Time: ", duration, unit)

    path = []
    while temp_node is not None:
        path.append(temp_node)
        temp_node = temp_node.parent

    path.reverse()

    print("==============")
    print("==== Path ====\n==============")
    for step in path:
        print(step)
        print("\"" + step.direction + "\"")
    print("==============")
    print("==============")

Heap:  [[7, 2, 4, 5, 0, 6, 8, 3, 1]]
Heap:  [[7, 2, 4, 5, 3, 6, 8, 0, 1], [7, 2, 4, 5, 6, 0, 8, 3, 1], [7, 2, 4, 0, 5, 6, 8, 3, 1], [7, 0, 4, 5, 2, 6, 8, 3, 1]]
Heap:  [[7, 2, 4, 0, 5, 6, 8, 3, 1], [7, 2, 4, 5, 6, 0, 8, 3, 1], [7, 0, 4, 5, 2, 6, 8, 3, 1], [7, 2, 4, 5, 3, 6, 0, 8, 1]]
Heap:  [[7, 2, 4, 5, 6, 0, 8, 3, 1], [7, 2, 4, 5, 3, 6, 0, 8, 1], [7, 0, 4, 5, 2, 6, 8, 3, 1], [7, 2, 4, 5, 3, 6, 8, 1, 0]]
Heap:  [[7, 2, 4, 5, 3, 6, 0, 8, 1], [0, 2, 4, 7, 5, 6, 8, 3, 1], [7, 2, 0, 5, 6, 4, 8, 3, 1], [7, 2, 4, 5, 3, 6, 8, 1, 0]]
Heap:  [[7, 2, 0, 5, 6, 4, 8, 3, 1], [0, 2, 4, 7, 5, 6, 8, 3, 1], [7, 2, 4, 5, 6, 1, 8, 3, 0], [7, 2, 4, 5, 3, 6, 8, 1, 0]]
Heap:  [[7, 2, 4, 5, 6, 1, 8, 3, 0], [0, 2, 4, 7, 5, 6, 8, 3, 1], [7, 0, 2, 5, 6, 4, 8, 3, 1], [7, 2, 4, 5, 3, 6, 8, 1, 0]]
Heap:  [[7, 0, 2, 5, 6, 4, 8, 3, 1], [0, 2, 4, 7, 5, 6, 8, 3, 1], [7, 2, 4, 0, 3, 6, 5, 8, 1], [7, 2, 4, 5, 3, 6, 8, 1, 0]]
Heap:  [[0, 2, 4, 7, 5, 6, 8, 3, 1], [7, 2, 4, 5, 3, 6, 8, 1, 0], [7, 2, 4, 0, 3, 6, 5, 8, 1], 

Heap:  [[2, 4, 6, 3, 5, 1, 7, 0, 8], [7, 1, 2, 3, 8, 4, 6, 5, 0], [3, 7, 2, 6, 1, 0, 5, 4, 8], [7, 2, 4, 1, 0, 3, 5, 8, 6]]
Heap:  [[3, 7, 2, 6, 1, 0, 5, 4, 8], [7, 1, 2, 3, 8, 4, 6, 5, 0], [2, 0, 4, 7, 1, 3, 6, 5, 8], [7, 2, 4, 1, 0, 3, 5, 8, 6]]
Heap:  [[2, 0, 4, 7, 1, 3, 6, 5, 8], [7, 1, 2, 3, 8, 4, 6, 5, 0], [2, 6, 5, 3, 0, 4, 7, 8, 1], [7, 2, 4, 1, 0, 3, 5, 8, 6]]
Heap:  [[2, 6, 5, 3, 0, 4, 7, 8, 1], [7, 1, 2, 3, 8, 4, 6, 5, 0], [2, 4, 0, 7, 1, 5, 8, 3, 6], [7, 2, 4, 1, 0, 3, 5, 8, 6]]
Heap:  [[2, 4, 0, 7, 1, 5, 8, 3, 6], [7, 1, 2, 3, 8, 4, 6, 5, 0], [6, 3, 2, 7, 0, 4, 5, 8, 1], [7, 2, 4, 1, 0, 3, 5, 8, 6]]
Heap:  [[6, 3, 2, 7, 0, 4, 5, 8, 1], [7, 1, 2, 3, 8, 4, 6, 5, 0], [7, 4, 1, 8, 5, 2, 6, 3, 0], [7, 2, 4, 1, 0, 3, 5, 8, 6]]
Heap:  [[7, 4, 1, 8, 5, 2, 6, 3, 0], [7, 1, 2, 3, 8, 4, 6, 5, 0], [2, 4, 6, 3, 5, 1, 0, 7, 8], [7, 2, 4, 1, 0, 3, 5, 8, 6]]
Heap:  [[2, 4, 6, 3, 5, 1, 0, 7, 8], [7, 1, 2, 3, 8, 4, 6, 5, 0], [2, 4, 0, 7, 1, 3, 6, 5, 8], [7, 2, 4, 1, 0, 3, 5, 8, 6]]
Heap:  [

Heap:  [[7, 6, 2, 3, 8, 4, 0, 5, 1], [5, 7, 4, 8, 0, 2, 3, 1, 6], [2, 4, 3, 1, 0, 5, 7, 8, 6], [3, 5, 2, 7, 4, 1, 6, 0, 8]]
Heap:  [[5, 7, 4, 8, 0, 2, 3, 1, 6], [3, 5, 2, 7, 4, 1, 6, 0, 8], [2, 4, 3, 1, 0, 5, 7, 8, 6], [7, 2, 5, 3, 0, 4, 8, 1, 6]]
Heap:  [[3, 5, 2, 7, 4, 1, 6, 0, 8], [7, 2, 5, 3, 0, 4, 8, 1, 6], [2, 4, 3, 1, 0, 5, 7, 8, 6], [2, 4, 0, 7, 1, 3, 5, 8, 6]]
Heap:  [[7, 2, 5, 3, 0, 4, 8, 1, 6], [2, 4, 0, 7, 1, 3, 5, 8, 6], [2, 4, 3, 1, 0, 5, 7, 8, 6], [7, 6, 2, 0, 5, 1, 8, 4, 3]]
Heap:  [[2, 4, 0, 7, 1, 3, 5, 8, 6], [7, 6, 2, 0, 5, 1, 8, 4, 3], [2, 4, 3, 1, 0, 5, 7, 8, 6], [7, 0, 2, 5, 6, 1, 8, 4, 3]]
Heap:  [[7, 6, 2, 0, 5, 1, 8, 4, 3], [7, 0, 2, 5, 6, 1, 8, 4, 3], [2, 4, 3, 1, 0, 5, 7, 8, 6], [7, 2, 4, 8, 0, 1, 6, 5, 3]]
Heap:  [[7, 0, 2, 5, 6, 1, 8, 4, 3], [7, 2, 4, 8, 0, 1, 6, 5, 3], [2, 4, 3, 1, 0, 5, 7, 8, 6], [5, 7, 4, 2, 0, 6, 8, 3, 1]]
Heap:  [[7, 2, 4, 8, 0, 1, 6, 5, 3], [5, 7, 4, 2, 0, 6, 8, 3, 1], [2, 4, 3, 1, 0, 5, 7, 8, 6], [7, 0, 2, 5, 1, 4, 3, 8, 6]]
Heap:  [

Heap:  [[5, 7, 2, 8, 4, 0, 6, 1, 3], [1, 0, 2, 3, 8, 4, 7, 5, 6], [7, 4, 6, 3, 1, 0, 5, 2, 8], [3, 2, 4, 7, 5, 1, 0, 6, 8]]
Heap:  [[7, 4, 6, 3, 1, 0, 5, 2, 8], [1, 0, 2, 3, 8, 4, 7, 5, 6], [1, 2, 4, 0, 7, 5, 8, 6, 3], [3, 2, 4, 7, 5, 1, 0, 6, 8]]
Heap:  [[1, 2, 4, 0, 7, 5, 8, 6, 3], [1, 0, 2, 3, 8, 4, 7, 5, 6], [6, 5, 2, 4, 1, 3, 7, 0, 8], [3, 2, 4, 7, 5, 1, 0, 6, 8]]
Heap:  [[6, 5, 2, 4, 1, 3, 7, 0, 8], [1, 0, 2, 3, 8, 4, 7, 5, 6], [2, 1, 5, 0, 4, 3, 7, 8, 6], [3, 2, 4, 7, 5, 1, 0, 6, 8]]
Heap:  [[2, 1, 5, 0, 4, 3, 7, 8, 6], [1, 0, 2, 3, 8, 4, 7, 5, 6], [5, 7, 2, 8, 4, 3, 6, 1, 0], [3, 2, 4, 7, 5, 1, 0, 6, 8]]
Heap:  [[5, 7, 2, 8, 4, 3, 6, 1, 0], [1, 0, 2, 3, 8, 4, 7, 5, 6], [2, 1, 5, 7, 0, 4, 8, 6, 3], [3, 2, 4, 7, 5, 1, 0, 6, 8]]
Heap:  [[2, 1, 5, 7, 0, 4, 8, 6, 3], [1, 0, 2, 3, 8, 4, 7, 5, 6], [7, 4, 3, 0, 1, 2, 5, 8, 6], [3, 2, 4, 7, 5, 1, 0, 6, 8]]
Heap:  [[7, 4, 3, 0, 1, 2, 5, 8, 6], [1, 0, 2, 3, 8, 4, 7, 5, 6], [7, 4, 3, 8, 5, 2, 0, 1, 6], [3, 2, 4, 7, 5, 1, 0, 6, 8]]
Heap:  [

Heap:  [[2, 0, 1, 7, 4, 6, 5, 8, 3], [1, 0, 2, 3, 8, 4, 7, 5, 6], [8, 2, 4, 3, 5, 6, 0, 7, 1], [3, 2, 4, 7, 5, 1, 0, 6, 8]]
Heap:  [[8, 2, 4, 3, 5, 6, 0, 7, 1], [1, 0, 2, 3, 8, 4, 7, 5, 6], [2, 4, 1, 0, 7, 3, 5, 6, 8], [3, 2, 4, 7, 5, 1, 0, 6, 8]]
Heap:  [[2, 4, 1, 0, 7, 3, 5, 6, 8], [1, 0, 2, 3, 8, 4, 7, 5, 6], [2, 7, 5, 3, 8, 4, 0, 1, 6], [3, 2, 4, 7, 5, 1, 0, 6, 8]]
Heap:  [[2, 7, 5, 3, 8, 4, 0, 1, 6], [1, 0, 2, 3, 8, 4, 7, 5, 6], [2, 7, 5, 8, 1, 4, 3, 6, 0], [3, 2, 4, 7, 5, 1, 0, 6, 8]]
Heap:  [[2, 7, 5, 8, 1, 4, 3, 6, 0], [1, 0, 2, 3, 8, 4, 7, 5, 6], [6, 3, 2, 7, 1, 0, 5, 4, 8], [3, 2, 4, 7, 5, 1, 0, 6, 8]]
Heap:  [[6, 3, 2, 7, 1, 0, 5, 4, 8], [1, 0, 2, 3, 8, 4, 7, 5, 6], [3, 5, 2, 7, 1, 0, 6, 4, 8], [3, 2, 4, 7, 5, 1, 0, 6, 8]]
Heap:  [[3, 5, 2, 7, 1, 0, 6, 4, 8], [1, 0, 2, 3, 8, 4, 7, 5, 6], [2, 1, 0, 7, 4, 6, 5, 8, 3], [3, 2, 4, 7, 5, 1, 0, 6, 8]]
Heap:  [[2, 1, 0, 7, 4, 6, 5, 8, 3], [1, 0, 2, 3, 8, 4, 7, 5, 6], [7, 2, 4, 6, 1, 5, 8, 0, 3], [3, 2, 4, 7, 5, 1, 0, 6, 8]]
Heap:  [

Heap:  [[7, 1, 2, 6, 0, 8, 3, 4, 5], [1, 3, 2, 5, 0, 4, 6, 7, 8], [2, 4, 1, 6, 7, 5, 0, 3, 8], [3, 2, 4, 7, 5, 1, 0, 6, 8]]
Heap:  [[1, 3, 2, 5, 0, 4, 6, 7, 8], [7, 1, 2, 4, 3, 0, 5, 8, 6], [2, 4, 1, 6, 7, 5, 0, 3, 8], [3, 2, 4, 7, 5, 1, 0, 6, 8]]
Heap:  [[7, 1, 2, 4, 3, 0, 5, 8, 6], [5, 2, 0, 3, 7, 8, 6, 1, 4], [2, 4, 1, 6, 7, 5, 0, 3, 8], [3, 2, 4, 7, 5, 1, 0, 6, 8]]
Heap:  [[5, 2, 0, 3, 7, 8, 6, 1, 4], [6, 2, 0, 4, 7, 5, 8, 3, 1], [2, 4, 1, 6, 7, 5, 0, 3, 8], [3, 2, 4, 7, 5, 1, 0, 6, 8]]
Heap:  [[6, 2, 0, 4, 7, 5, 8, 3, 1], [4, 2, 0, 7, 6, 5, 8, 3, 1], [2, 4, 1, 6, 7, 5, 0, 3, 8], [3, 2, 4, 7, 5, 1, 0, 6, 8]]
Heap:  [[4, 2, 0, 7, 6, 5, 8, 3, 1], [0, 3, 4, 2, 7, 6, 5, 8, 1], [2, 4, 1, 6, 7, 5, 0, 3, 8], [3, 2, 4, 7, 5, 1, 0, 6, 8]]
Heap:  [[0, 3, 4, 2, 7, 6, 5, 8, 1], [3, 1, 2, 5, 7, 8, 6, 4, 0], [2, 4, 1, 6, 7, 5, 0, 3, 8], [3, 2, 4, 7, 5, 1, 0, 6, 8]]
Heap:  [[3, 1, 2, 5, 7, 8, 6, 4, 0], [7, 2, 4, 5, 3, 8, 1, 6, 0], [2, 4, 1, 6, 7, 5, 0, 3, 8], [3, 2, 4, 7, 5, 1, 0, 6, 8]]
Heap:  [